# QUBO get minimal value
from article [Finding the Smallest Number of an Array with Quantum Computer](https://medium.com/@aniruddha.connect/finding-smallest-number-of-an-array-with-quantum-computer-824386348fe0)

**Doc about QAOA**
- https://qiskit.org/ecosystem/optimization/stubs/qiskit_optimization.QuadraticProgram.html#quadraticprogram
- https://qiskit.org/ecosystem/optimization/tutorials/03_minimum_eigen_optimizer.html

In [7]:
from qiskit_algorithms.utils import algorithm_globals
from qiskit_algorithms.minimum_eigensolvers import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    GroverOptimizer,
    MinimumEigenOptimizer,
)
from qiskit_optimization import QuadraticProgram
from itertools import combinations
from typing import List, Tuple
import numpy as np
import emoji
import math

backend_sim = Sampler()
algorithm_globals.random_seed = 42

In [8]:
dataset = [1, 2, 3, -4]
params = ["a", "b", "c", "d"]    # 1 param = 1 data point
constant = 0
gen_coef = 1

In [9]:
qubo = QuadraticProgram(name="Get minimal value.")
for param in params:
    qubo.binary_var(param)

qubo.minimize(
    constant=constant,
    linear=dataset,
    quadratic={(x, y): gen_coef for x, y in combinations(params, 2)}
)
qubo.linear_constraint(
    name="2_minimals",
    linear={param: 1 for param in params},
    sense="==",
    rhs=2
)

print(qubo.prettyprint())

Problem name: Get minimal value.

Minimize
  a*b + a*c + a*d + b*c + b*d + c*d + a + 2*b + 3*c - 4*d

Subject to
  Linear constraints (1)
    a + b + c + d == 2  '2_minimals'

  Binary variables (4)
    a b c d



## Classical resolution
- Result doc : https://qiskit.org/ecosystem/optimization/stubs/qiskit_optimization.algorithms.OptimizationResult.html

In [10]:
exact_mes = NumPyMinimumEigensolver()
exact = MinimumEigenOptimizer(exact_mes)
exact_result = exact.solve(qubo)
print(exact_result.prettyprint())

dataset_sort = sorted(dataset)
mini = dataset.index(dataset_sort[0])
almost_mini = dataset.index(dataset_sort[1])
if exact_result.x[mini] == 1 and exact_result.x[almost_mini] == 1:
    print(emoji.emojize("\nSame as the theorical result. \nCongratulations :party_popper: !!"))
else:
    print(emoji.emojize("\nNot passing :cross_mark:"))

objective function value: -2.0
variable values: a=1.0, b=0.0, c=0.0, d=1.0
status: SUCCESS

Same as the theorical result. 
Congratulations 🎉 !!


## QAOA

In [11]:
qaoa_mes = QAOA(sampler=backend_sim, optimizer=COBYLA(), initial_point=[0.0, 0.0])
qaoa = MinimumEigenOptimizer(qaoa_mes)
qaoa_result = qaoa.solve(qubo)
print(qaoa_result.prettyprint())

if qaoa_result.prettyprint() == exact_result.prettyprint():
    print(emoji.emojize("\nSame as the classical result. \nCongratulations :party_popper: !!"))
else:
    print(emoji.emojize("\nNot passing :cross_mark:"))

objective function value: -2.0
variable values: a=1.0, b=0.0, c=0.0, d=1.0
status: SUCCESS

Same as the classical result. 
Congratulations 🎉 !!


## Grover

In [12]:
nb_qubits = math.ceil(math.log2(len(dataset)))
nb_iter = math.ceil((np.pi/len(dataset))*(math.sqrt(2**len(dataset))))
grover_optimizer = GroverOptimizer(nb_qubits, num_iterations=nb_iter, sampler=backend_sim)
grover_result = grover_optimizer.solve(qubo)
print(grover_result.prettyprint())

if grover_result.prettyprint() == exact_result.prettyprint():
    print(emoji.emojize("\nSame as the classical result. \nCongratulations :party_popper: !!"))
else:
    print(emoji.emojize("\nNot passing :cross_mark:"))
if grover_result.status.value == 2:
    print(emoji.emojize(":warning: seems not usable for most of the QUBO problems"))

objective function value: -1.0
variable values: a=0.0, b=1.0, c=0.0, d=1.0
status: SUCCESS

Not passing ❌


## Data analyse
Resources for plot :
- https://qiskit.org/documentation/tutorials/algorithms/05_qaoa.html
- https://qiskit.org/ecosystem/optimization/tutorials/06_examples_max_cut_and_tsp.html

In [13]:
...

Ellipsis

---